In [10]:
import spi
import rioxarray as rx
import pfast

In [4]:
path=r"D:\Package_Development\spydis\data\MODIS_Precipitation_VN.tif"
ds=rx.open_rasterio(path)

In [11]:
ds=pfast.time_dimension(ds,2000,2,1)

In [21]:
ds

<xarray.DataArray (time: 263, y: 46, x: 51)>
array([[[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       ...,

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]],

       [[nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan],
        ...,
        [nan, nan, ..., nan, nan],
        [nan, nan, ..., nan, nan]]], dtype=float32)
Coordinates:
  * time         (time) datetime64[ns] 2000-02-01 2000-03-01 ... 2021-12-01
  * x            (x) float64 105.3 105.3 105.3 105.4 ... 105.8 105.8 105.8 105.8
  * y            (y) float64 21.56 21.56 21.55 21.54 ... 21.19 21.18 21.17 21.16
    spatial_ref  int32 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

In [ ]:
def SPI(ds, timescale,dimname):
    """Function includes three input parameters:
    data: A xarray-like data or raster data reads as xarray or numpy-like array
    time: Timescale for calculating SPI. It can be 1 month timescale, three month timescale, etc.
    dim: A dimension of time
    """
    if isinstance(ds,(xr.core.dataarray.DataArray)):
        pass
    else:
        print("Please make sure the data is xarray or numpy-like data")
    if isinstance(timescale, int):
        if timescale>0:
            pass
    else:
        print("Please make sure timescale is positive and integer")
    # Calculate mean moving average
    ds_ma=ds.rolling({dimname:timescale}, center=False).sum(dim=dimname) 
    # Calculate natural log of miving average
    ds_log=np.log(ds_ma)
    ds_log=ds_log.where(np.isinf(ds_log)==False) # Change infinity ==nan
    # Calculate overall mean of moving average
    ds_mu=ds_ma.mean(dim=dimname)
    # Calculate summation of log
    ds_sum=ds_log.sum(dim=dimname)
    # Computing essentials for gamma distribution
    n=ds_log[timescale-1:,:,:].count(dim=dimname)
    A=np.log(ds_mu)-(ds_sum/n)
    alpha=(1/(4*A))*(1+(1+((4*A)/3))**0.5) # Computing alpha
    beta = ds_mu/alpha    # Computing beta (scale)
    # Gamma distribution
    gamma_func=lambda data, a, scale: st.gamma.cdf(data,a=a,scale=scale)
    gamma=xr.apply_ufunc(gamma_func, ds_ma, alpha,beta)
    # Standardized precipitation index (Inverse of CDF)
    norm_inverse=lambda data: st.norm.ppf(data,loc=0,scale=1)
    norm_spi=xr.apply_ufunc(norm_inverse, gamma)
    return norm_spi


# In[ ]:




